# ETL - LOAD

In [52]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import DATABASE_URI
import datetime

In [53]:
engine = create_engine(DATABASE_URI)
conn = engine.connect()

In [54]:
FinalMoviesBudget = pd.read_csv('FinalMoviesBudget.csv')
FinalMoviesBudget.head()

,id,title,vote_count,vote_average,revenue,runtime,popularity,ReleaseDate,ProductionBudget,DomesticGross,WorldwideGross
0,299534,Avengers: Endgame,15589,8.3,2797800564,181.0,193.659,2019-04-23,400000000,858373000,2797800564
1,1865,Pirates of the Caribbean: On Stranger Tides,10375,6.5,1045713802,136.0,79.193,2011-05-20,379000000,241063875,1045663875
2,99861,Avengers: Age of Ultron,16755,7.3,1405403694,141.0,85.183,2015-04-22,365000000,459005868,1396099202
3,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850,2018-04-25,300000000,678815482,2044540523
4,141052,Justice League,9747,6.2,657924295,120.0,112.464,2017-11-13,300000000,229024295,655945209


In [55]:
FinalMoviesBudget['ReleaseDate'] = pd.to_datetime(FinalMoviesBudget['ReleaseDate'])


In [56]:
FinalMoviesBudget.columns = FinalMoviesBudget.columns.str.lower()

In [60]:
netflix_movies_revenue = pd.read_csv('netflix_movies_revenue.csv')
netflix_movies_revenue.head()

,id,title,country,release_year,rating,revenue,vote_count,vote_average,runtime,popularity
0,601131,Norm of the North: King Sized Adventure,United States,2019,TV-PG,1442504,4,7.6,90.0,8.420
1,415722,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,0,2,5.0,94.0,0.600
2,262543,Automata,Bulgaria,2014,R,0,1171,5.8,110.0,25.466
3,262338,Good People,United States,2014,R,0,324,5.5,90.0,28.195
4,474599,Joaquín Reyes: Una y no más,unknown,2017,TV-MA,0,1,6.0,77.0,0.600


In [61]:
netflix_movies_revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3148 entries, 0 to 3147
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3148 non-null   int64  
 1   title         3148 non-null   object 
 2   country       3148 non-null   object 
 3   release_year  3148 non-null   int64  
 4   rating        3148 non-null   object 
 5   revenue       3148 non-null   int64  
 6   vote_count    3148 non-null   int64  
 7   vote_average  3148 non-null   float64
 8   runtime       3148 non-null   float64
 9   popularity    3148 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 246.1+ KB


In [58]:
# Check tables if loaded
engine.table_names()

['finalmoviesbudget', 'netflix_movies_revenue']

In [59]:
# Load finalmoviesbudget to the database
FinalMoviesBudget.to_sql(name = 'finalmoviesbudget', con = engine, if_exists = 'append', index = False)

In [62]:
netflix_movies_revenue.to_sql(name = 'netflix_movies_revenue', con = engine, if_exists = 'append', index = False)

In [63]:
# Query netflix_movies_revenue to confirm tables are loaded
pd.read_sql_query('SELECT * FROM finalmoviesbudget', con = engine).head()


,id,title,vote_count,vote_average,revenue,runtime,popularity,releasedate,productionbudget,domesticgross,worldwidegross
0,299534,Avengers: Endgame,15589,8.3,2797800564,181.0,193.659,2019-04-23,400000000,858373000,2797800564
1,1865,Pirates of the Caribbean: On Stranger Tides,10375,6.5,1045713802,136.0,79.193,2011-05-20,379000000,241063875,1045663875
2,99861,Avengers: Age of Ultron,16755,7.3,1405403694,141.0,85.183,2015-04-22,365000000,459005868,1396099202
3,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850,2018-04-25,300000000,678815482,2044540523
4,141052,Justice League,9747,6.2,657924295,120.0,112.464,2017-11-13,300000000,229024295,655945209


In [64]:
pd.read_sql_query('SELECT * FROM netflix_movies_revenue', con = engine).head()

,id,title,country,release_year,rating,revenue,vote_count,vote_average,runtime,popularity
0,601131,Norm of the North: King Sized Adventure,United States,2019,TV-PG,1442504,4,7.6,90.0,8.420
1,415722,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,0,2,5.0,94.0,0.600
2,262543,Automata,Bulgaria,2014,R,0,1171,5.8,110.0,25.466
3,262338,Good People,United States,2014,R,0,324,5.5,90.0,28.195
4,474599,Joaquín Reyes: Una y no más,unknown,2017,TV-MA,0,1,6.0,77.0,0.600


In [49]:
conn.close()